In [9]:
import win32gui
import win32con
import win32api

In [39]:
def EnumWindows():
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumWindows(_MyCallback, windows)
    return windows

In [16]:
def EnumChildWindows(hwnd):
    windows = {}
    def _MyCallback( hwnd, extra ):
        windows = extra
        temp={}
        temp['hex'] = hex(hwnd)
        temp['ClassName'] = win32gui.GetClassName(hwnd)
        temp['WindowText'] = win32gui.GetWindowText(hwnd)
        windows[hwnd] = temp
    win32gui.EnumChildWindows(hwnd,_MyCallback, windows)
    return windows

In [21]:
childs = EnumChildWindows('1444364')
childs

{395806: {'hex': '0x60a1e',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 854514: {'hex': '0xd09f2',
  'ClassName': 'ApplicationFrameTitleBarWindow',
  'WindowText': ''},
 526886: {'hex': '0x80a26',
  'ClassName': 'ApplicationFrameInputSinkWindow',
  'WindowText': ''}}

In [30]:
child_text = win32gui.GetWindowText('1444364')
print(child_text)

设置


In [50]:
# 搜索子窗口
# 枚举子窗口
hwndChildList = []
win32gui.EnumChildWindows('1444364', lambda hwnd, param: param.append(hwnd),  hwndChildList)
hwndChildList

[395806, 854514, 526886]

In [87]:
import cv2
import numpy as np
from ctypes import windll
import win32gui
import win32ui
 
 
def capture_win_alt(hwnd=None, program_windowTitle=None, program_className=None,img_name='temp.png'):
    # Adapted from https://stackoverflow.com/questions/19695214/screenshot-of-inactive-window-printwindow-win32gui
    if hwnd is None:
        windll.user32.SetProcessDPIAware()
        windows = EnumWindows()
        for key,item in windows.items() :
            if item['ClassName'] == program_className:
                if program_windowTitle != None:
                    if program_windowTitle in item['WindowText'] or program_windowTitle == item['WindowText']:
                        program_windowTitle = item['WindowText']
                        hwnd = key
                        print(key,item)
                        break;
                    else:
                        hwnd = key
                        print(key,item)
                        break;
    if not hwnd:
        hwnd = win32gui.FindWindow(program_className, program_windowTitle)
        # hwnd = win32gui.FindWindowEx(0, 0, program_className, program_windowTitle)
    if not hwnd:
        return None
    left, top, right, bottom = win32gui.GetClientRect(hwnd)
    w = right - left
    h = bottom - top
    hwnd_dc = win32gui.GetWindowDC(hwnd)
    mfc_dc = win32ui.CreateDCFromHandle(hwnd_dc)
    save_dc = mfc_dc.CreateCompatibleDC()
    bitmap = win32ui.CreateBitmap()
    bitmap.CreateCompatibleBitmap(mfc_dc, w, h)
    save_dc.SelectObject(bitmap)
    # If Special K is running, this number is 3. If not, 1
    result = windll.user32.PrintWindow(hwnd, save_dc.GetSafeHdc(), 3)
    bmpinfo = bitmap.GetInfo()
    bmpstr = bitmap.GetBitmapBits(True)
    img = np.frombuffer(bmpstr, dtype=np.uint8).reshape((bmpinfo["bmHeight"], bmpinfo["bmWidth"], 4))
    img = np.ascontiguousarray(img)[..., :-1]  # make image C_CONTIGUOUS and drop alpha channel
    if not result:  # result should be 1
        win32gui.DeleteObject(bitmap.GetHandle())
        save_dc.DeleteDC()
        mfc_dc.DeleteDC()
        win32gui.ReleaseDC(hwnd, hwnd_dc)
        raise RuntimeError(f"Unable to acquire screenshot! Result: {result}")
    cv2.imwrite(img_name, img)
    return img,w,h,hwnd

In [94]:
from cnocr import CnOcr


img,w,h,hwnd = capture_win_alt(program_className='ApplicationFrameWindow',program_windowTitle='')

ocr = CnOcr() 
result = ocr.ocr(img)

# 具体参数设置参考：https://cnocr.readthedocs.io/zh/latest/usage/


723436 {'hex': '0xb09ec', 'ClassName': 'ApplicationFrameWindow', 'WindowText': '设置'}


In [6]:
'xxxxxx'[:2]

'xx'

In [97]:
from cnocr import CnOcr

ocr = CnOcr()
img,x,y,hwnd = capture_win_alt(program_className='ApplicationFrameWindow',program_windowTitle='')

step_key = ['时间和语言']
text_positions = []
for item in ocr.ocr(img):
    if item['score'] > 0.9:
        if item['text'] in step_key:
            temp = {
                'text': item['text'],
                'position': [
                    (item['position'][0][0]+item['position'][2][0])/2, 
                    (item['position'][0][1]+item['position'][2][1])/2
                             ]
            }
            temp['position'][0] = temp['position'][0] / x
            temp['position'][1] = temp['position'][1] / y
            text_positions.append(temp)


723436 {'hex': '0xb09ec', 'ClassName': 'ApplicationFrameWindow', 'WindowText': '设置'}


In [98]:
left, top, right, bottom = win32gui.GetWindowRect(hwnd)
print(left, top, right, bottom)
cy = int(text_positions[0]['position'][1] * (bottom - top) + top)
cx = int(text_positions[0]['position'][0] * (right - left) + left)
print(cx,cy)
long_position = win32api.MAKELONG(cx, cy)#模拟鼠标指针 传送到指定坐标
win32api.SendMessage(hwnd, win32con.WM_LBUTTONDOWN, win32con.MK_LBUTTON, long_position)#模拟鼠标按下
win32api.SendMessage(hwnd, win32con.WM_LBUTTONUP, win32con.MK_LBUTTON, long_position)#模拟鼠标弹起

19 10 1317 1018
114 508


0

In [91]:
text_positions

[{'text': '时间和语言', 'position': array([     0.0945,     0.38612])}]

In [76]:
text_positions

[{'text': '时间和语言', 'position': array([      0.094,     0.38612])}]

In [1]:
import winreg

def get_window_software(hive, flag):
    """
    get windows all installed software
    """
    # winreg.ConnectRegistry(None, hive): 连接注册表; hive：windows下的HKEY_常量
    aReg = winreg.ConnectRegistry(None, hive)
    # winreg.OpenKey(reg, regedit_path, 0, reg_type):操作注册表（读取，写入等）
    # reg: 连接的注册表; regedit_path:注册表的路径; reg_type:需要对注册表的操作类型
    aKey = winreg.OpenKey(aReg, r"SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall",
                          0, winreg.KEY_READ | flag)
    # winreg.QueryInfoKey(key):获取注册表的相关信息; 返回一个元组：(此注册表的子key数量， 0， 自从1600.1.1上次修改时间（纳秒）)key:打开的注册表
    count_subkey = winreg.QueryInfoKey(aKey)[0]
    print(f"count_subkey:{count_subkey}")
    software_list = []
    for i in range(count_subkey):
        software = {}
        try:
            asubkey_name = winreg.EnumKey(aKey, i)
            asubkey = winreg.OpenKey(aKey, asubkey_name)
            # 根据键名查询对应值
            software['name'] = winreg.QueryValueEx(asubkey, "DisplayName")[0]
            try:
                software['version'] = winreg.QueryValueEx(asubkey, "DisplayVersion")[0]
            except EnvironmentError:
                software['version'] = 'undefined'
            try:
                software['publisher'] = winreg.QueryValueEx(asubkey, "Publisher")[0]
            except EnvironmentError:
                software['publisher'] = 'undefined'
            software_list.append(software)
        except EnvironmentError:
            continue

    return software_list

software_list = get_window_software(winreg.HKEY_LOCAL_MACHINE, winreg.KEY_WOW64_32KEY) + get_window_software(winreg.HKEY_LOCAL_MACHINE, winreg.KEY_WOW64_64KEY) + get_window_software(winreg.HKEY_CURRENT_USER, 0)
print(software_list)
for software in software_list:
    print('Name=%s, Version=%s, Publisher=%s' % (software['name'], software['version'], software['publisher']))
print('Number of installed apps: %s' % len(software_list))


count_subkey:275
count_subkey:293
count_subkey:16
[{'name': '360安全卫士', 'version': '13.0.0.2007', 'publisher': '360安全中心'}, {'name': 'Visual Studio 生成工具 2019', 'version': '16.11.34', 'publisher': 'Microsoft Corporation'}, {'name': '5G网联无人机地面站', 'version': '1.3.0.6', 'publisher': '天宇经纬（北京）科技有限公司'}, {'name': '暴雪战网', 'version': 'undefined', 'publisher': 'Blizzard Entertainment'}, {'name': '钉钉', 'version': '7.6.25-Release.102910812', 'publisher': 'Alibaba (China) Network Technology Co.,Ltd.'}, {'name': 'DirectX修复工具 4.0', 'version': '4.0', 'publisher': 'undefined'}, {'name': 'Visual Studio Community 2022', 'version': '17.9.5', 'publisher': 'Microsoft Corporation'}, {'name': 'GW-SOUL', 'version': '1.0.5.3', 'publisher': 'Galaxy Microsystems Ltd.'}, {'name': 'IntelliJ IDEA 2023.2.3', 'version': '232.10072.27', 'publisher': 'JetBrains s.r.o.'}, {'name': 'letsvpn', 'version': '3.11.0', 'publisher': 'Lets'}, {'name': 'Microsoft Edge', 'version': '131.0.2903.99', 'publisher': 'Microsoft Corporation